In [5]:
import os
#import csv
import math
import random
import operator
import string
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
#from bs4 import BeautifulSoup
%matplotlib inline
#import xgboost as xgb

In [25]:
root = 'C:\\Users\\NUREFSHINE\\Desktop\\2019Spring-3rdYear-2ndSemester\\CS464-2-Introduction to Machine Learning\\ML-proje\\'
csv_path = os.path.join(root, 'hatespeech_text_abusive.xlsx')
df = pd.read_excel(csv_path, header = None)
df.rename(columns={0:'tweet', 1:'label'}, inplace=True)
print(df.head(10))

my_labels = ['hateful', 'normal']
len(df)

                                               tweet    label
0  RT @Papapishu: Man it would fucking rule if we...  hateful
1  It is time to draw close to Him &#128591;&#127...   normal
2  if you notice me start to act different or dis...   normal
3  Forget unfollowers, I believe in growing. 7 ne...   normal
4  RT @Vitiligoprince: Hate Being sexually Frustr...  hateful
5  Topped the group in TGP Disc Jam Season 2! Ont...   normal
6  That daily baby aspirin for your #heart just m...   normal
7  I liked a @YouTube video from @mattshea https:...   normal
8  RT @LestuhGang_: If your fucking up &amp; your...  hateful
9  @Move_Fwd give up. You've lost. You will not c...   normal


85966

In [55]:
"bitch" in words.words()

True

In [43]:
for i in range(15):
    print_plot(i)

man would fucking rule party perpetual warfare
Label: hateful
time draw close father draw near always
Label: normal
notice start act different distant something notice difference act
Label: normal
forget believe growing new last day via
Label: normal
hate sexually like fuck ion fuck anybody
Label: hateful
topped group disc jam season onto
Label: normal
daily baby aspirin might colon
Label: normal
video blue army coming ancient warfare
Label: normal
fucking dont tell fucking
Label: hateful
give lost convince one iota read conspiracy
Label: normal
one way fucking threw candy ever
Label: hateful
fucking
Label: hateful
struggle remember song days remember nearly
Label: normal
nasty ass freak like
Label: hateful
tried new cure common cold
Label: normal


In [3]:
def print_plot(index):
    example = df[df.index == index][['tweet', 'label']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Label:', example[1])

In [56]:
STOPWORDS = stopwords.words('english')
STOPWORDS.append("rt")

WORDS = set(nltk.corpus.words.words())
WORDS.update({"fuck", "fucking"})

def clean_text():
    df["tweet"] = df["tweet"].apply(lambda x: x.lower())
    df["tweet"] = [re.sub('(@[^\s]+)|(#[^\s]+)', '', tweet) for tweet in df["tweet"]]
    df["tweet"] = [re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet) for tweet in df["tweet"]]
    df["tweet"] = [re.sub('(\'[^\s]+)|(&[^\s]+)','',tweet) for tweet in df["tweet"]]
    df["tweet"] = [re.sub('[^\w\s/:%.,_-]','',tweet) for tweet in df["tweet"]]
    df["tweet"] = df["tweet"].apply(lambda tweet: tweet.translate(str.maketrans('', '', string.punctuation)))
    df["tweet"] = df["tweet"].apply(lambda tweet: tweet.translate(str.maketrans('', '', "0123456789❤♀️♥⚽️《")) )
    df["tweet"] = df["tweet"].str.split(' ').apply(lambda tweet: ' '.join(k for k in tweet if k not in STOPWORDS))
    df["tweet"] = df["tweet"].str.split(' ').apply(lambda tweet: ' '.join(k for k in tweet if k in WORDS))
    df["tweet"] = df["tweet"].str.replace(' +', ' ', case=False)
    df["tweet"] = df["tweet"].str.strip()
    df["tweet"].replace('', np.nan, inplace=True)
    df.dropna(subset=["tweet"], inplace=True)
      
clean_text()
    
print(df.head(10))
print_plot(3)
len(df)

                                               tweet    label
0     man would fucking rule party perpetual warfare  hateful
1            time draw close father draw near always   normal
2  notice start act different distant something n...   normal
3            forget believe growing new last day via   normal
4           hate sexually like fuck ion fuck anybody  hateful
5                  topped group disc jam season onto   normal
6                     daily baby aspirin might colon   normal
7             video blue army coming ancient warfare   normal
8                          fucking dont tell fucking  hateful
9        give lost convince one iota read conspiracy   normal
forget believe growing new last day via
Label: normal


84865

In [44]:
from gensim.models import Word2Vec

word2vec_path = os.path.join(root, 'GoogleNews-vectors-negative300.bin.gz')
wv = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)
wv.init_sims(replace=True)

In [45]:
from itertools import islice
list(islice(wv.vocab, 13030, 13050))

['Memorial_Hospital',
 'Seniors',
 'memorandum',
 'elephant',
 'Trump',
 'Census',
 'pilgrims',
 'De',
 'Dogs',
 '###-####_ext',
 'chaotic',
 'forgive',
 'scholar',
 'Lottery',
 'decreasing',
 'Supervisor',
 'fundamentally',
 'Fitness',
 'abundance',
 'Hold']

In [46]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [57]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    
train, test = train_test_split(df, test_size=0.3, random_state = 42)

test_tokenized = test.apply(lambda r: w2v_tokenize_text(r['tweet']), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r['tweet']), axis=1).values

X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

c:\users\nurefshine\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  


In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_word_average, train['label'])
y_pred = logreg.predict(X_test_word_average)

print (y_pred)

print('accuracy %s' % accuracy_score(y_pred, test.label))
print(classification_report(test.label, y_pred, target_names = my_labels))

c:\users\nurefshine\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


['normal' 'hateful' 'hateful' ... 'normal' 'hateful' 'hateful']
accuracy 0.8858208955223881
              precision    recall  f1-score   support

     hateful       0.89      0.79      0.84      9403
      normal       0.88      0.94      0.91     16057

   micro avg       0.89      0.89      0.89     25460
   macro avg       0.89      0.87      0.87     25460
weighted avg       0.89      0.89      0.88     25460

